In [1]:
import pandas as pd

In [2]:
df = pd.read_excel('adultData.xlsx', na_values="?")

In [3]:
df.head()

age         workclass  fnlwgt  education  education-num  \
0   50  Self-emp-not-inc   83311  Bachelors             13   
1   38           Private  215646    HS-grad              9   
2   53           Private  234721       11th              7   
3   28           Private  338409  Bachelors             13   
4   37           Private  284582    Masters             14   

       marital-status         occupation   relationship   race     sex  \
0  Married-civ-spouse    Exec-managerial        Husband  White    Male   
1            Divorced  Handlers-cleaners  Not-in-family  White    Male   
2  Married-civ-spouse  Handlers-cleaners        Husband  Black    Male   
3  Married-civ-spouse     Prof-specialty           Wife  Black  Female   
4  Married-civ-spouse    Exec-managerial           Wife  White  Female   

   capital-gain  capital-loss  hours-per-week native-country salary  
0             0             0              13  United-States  <=50K  
1             0             0              40  United-States  <=50K  
2             0             0              40  United-States  <=50K  
3             0             0              40           Cuba  <=50K  
4             0             0              40  United-States  <=50K

In [4]:
df.columns = df.columns.str.strip()

### How many people of each race are represented in this dataset? This should be a Pandas series with race names as the index labels. (race column)

In [5]:
count_race = df['race'].value_counts()
print(count_race)

race
White                 27815
Black                  3124
Asian-Pac-Islander     1039
Amer-Indian-Eskimo      311
Other                   271
Name: count, dtype: int64


### What is the average age of men?

In [6]:
males = df[df['sex'].str.contains('Male')]

average_age = males['age'].mean()
print(f"Average age of Males: {average_age:.2f}")

Average age of Males: 39.43


### What is the percentage of people who have a Bachelor's degree?

In [7]:
bachelor = df[df['education'].str.contains('Bachelors')].shape[0]

total_people = df.shape[0]

percentage = (bachelor / total_people) * 100

print(f"Percentage of people who have Bachelors: {percentage:.2f}%")

Percentage of people who have Bachelors: 16.44%


### What percentage of people with advanced education (Bachelors, Masters, or Doctorate) make more than 50K?

In [8]:
education = df[df['education'].str.contains('Bachelors|Masters|Doctorate', case=False, na=False)]
print(education.shape[0])

more50k = education[education['salary'].str.contains('>50k', case=False)]
print(more50k.shape[0])

percentage_more_50k = (more50k.shape[0] / education.shape[0]) * 100

print(f"Percentage of People with advanced education, making more than 50k: {percentage_more_50k:.2f}%")


7489
3485
Percentage of People with advanced education, making more than 50k: 46.53%


### What percentage of people without advanced education make more than 50K?

In [9]:
# Identifica todos que nao possuem alta escolaridade, atençao que estou usando "~", para negativar a sentença
low_education = df[~df['education'].str.contains('Bachelors|Masters|Doctorate', case=False, na=False)]
print(low_education.shape[0])

# Identica as pessoas que ganham mais de 50k com baixa escolaridade
more50k_low_education  = low_education[low_education['salary'].str.contains('>50k', case=False)]
print(more50k_low_education .shape[0])

# Faz o caculo da porcentagem
percentage_more_50k_low_education  = (more50k_low_education.shape[0] / low_education .shape[0]) * 100
print(f"Percentage of People with low education, making more than 50k: {percentage_more_50k_low_education:.2f}%")



25071
4356
Percentage of People with low education, making more than 50k: 17.37%


### What is the minimum number of hours a person works per week?


In [10]:
min_hours = df['hours-per-week'].min()
print(f"Minimum hours works per week: {min_hours} hours")

Minimum hours works per week: 1 hours


### What percentage of the people who work the minimum number of hours per week have a salary of more than 50K?

In [11]:
# Identifica o menor número de horas trabalhadas por semana
min_hours = df['hours-per-week'].min()
print(min_hours)

# Filtra as pessoas que trabalham o mínimo de horas
min_hours_workers = df[df['hours-per-week'] == min_hours]
print(min_hours_workers.shape[0])

# Conta quantas dessas pessoas ganham mais de 50k
min_hours_high_earners  = min_hours_workers[min_hours_workers['salary'].str.contains('>50k', case=False)]
print(min_hours_high_earners .shape[0])

# Calcula a porcentagem
percentage = (len(min_hours_high_earners) / len(min_hours_workers)) * 100 if len(min_hours_workers) > 0 else 0
print(f"Percentage of people who work the minimum hours and make more than 50K: {percentage:.2f}%")


1
20
2
Percentage of people who work the minimum hours and make more than 50K: 10.00%


### What country has the highest percentage of people that earn >50K and what is that percentage?

In [12]:
from IPython.display import display

# Pessoas que ganham mais de 50k
more50k = df[df['salary'].str.contains('>50k', case=False)]

# Pessoas que ganham mais de 50k por pais
count_more50k = more50k['native-country'].value_counts()

# Conta o total de pessoas por pais
total_by_country = df['native-country'].value_counts()

# Calcula a porcentagem
percentage = (count_more50k / total_by_country) * 100

# Cria um novo dataFrame com as porcentagens para visualizaçao
percentage_df = percentage.reset_index()
percentage_df.columns = ['Country', 'Percentage']
sorted_percentage_df = percentage_df.dropna().sort_values(by='Percentage', ascending=False)
top10 = sorted_percentage_df.head(10)
display(top10)

# Encontra o pais com maior porcentagem
highest_percentage_country = percentage.idxmax()
highest_percentage = percentage.max()

print(f"O país com a maior porcentagem de pessoas que ganham mais de 50K é {highest_percentage_country} com {highest_percentage:.2f}%.")


Country  Percentage
19        Iran   41.860465
9       France   41.379310
18       India   40.000000
35      Taiwan   39.215686
23       Japan   38.709677
40  Yugoslavia   37.500000
0     Cambodia   36.842105
21       Italy   34.246575
8      England   33.333333
1       Canada   32.231405

O país com a maior porcentagem de pessoas que ganham mais de 50K é Iran com 41.86%.


### Identify the most popular occupation for those who earn >50K in India.


In [13]:
from IPython.display import display

# Pessoas com que ganham mais de 50k na india
india_more50k = df[(df['salary'].str.contains('>50k', case=False)) & (df['native-country'].str.contains("India", case=False))]

# Conta o numero de trabalhos que aparecem na lista
filtered_job = india_more50k['occupation'].value_counts()

# Cria um novo DataFrame
filtered_job_df = filtered_job.reset_index()
filtered_job_df.columns = ['Ocuppation', 'Count']
display(filtered_job_df.dropna())

# Pega o Nome do trabalho
name_occupation = filtered_job.idxmax()
# O numero de vezes que aparece
occupation_most_popular = filtered_job.max()

Ocuppation  Count
0    Prof-specialty     25
1   Exec-managerial      8
2     Other-service      2
3      Tech-support      2
4  Transport-moving      1
5             Sales      1
6      Adm-clerical      1